In [2]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [27]:
def solve_battleships(I, J, R, C, T, S, L):
    K = 7
    numShips = len(L)
    model = gp.Model('Battleships')
    # Define decision variables
    x = model.addVars(I, J, K, vtype=GRB.BINARY, name="x")  # Cell assignment
    s = model.addVars(I, J, numShips, vtype=GRB.BINARY, name="s")  # Ship starts


    #constraint 1
    for i in range(I):
        for j in range(J):
            model.addConstr(gp.quicksum(
                x[i, j, k] for k in range(K)
            ) == 1)

    #constraint 2
    for i in range(I):
        for j in range(J):
            for k in range(K):
                if T[i, j, k] == 1:
                    model.addConstr(x[i, j, k] == 1)

    #constraint 3
    for i in range(I):
        model.addConstr(
            gp.quicksum(x[i, j, k] for j in range(J) for k in range(1, K)) == R[i]
        )

    #constraint 4
    for j in range(J):
        model.addConstr(
            gp.quicksum(x[i, j, k] for i in range(I) for k in range(1, K)) == C[j]
        )

    #constraint 5
    for i in range(I):
        for j in range(J):
            model.addConstr(s[i, j, 0] == x[i, j, 6])
           
  
    #constraint 6
    for i in range(I):
        for j in range(J):
            for l in range(1,numShips):
                if j + L[l] -1 < J and i + L[l] - 1 < I:  # Ensure indices stay in range
                    sum1 = sum(x[i, c, 5] for c in range(j+1, j+L[l]-2))
                    sum2 = sum(x[r, j, 5] for r in range(i+1, i+L[l]-2))
                    model.addConstr(
                        L[l]*s[i, j, l] <= x[i, j, 1] + x[i, j+L[l]-1, 2] + sum1 + x[i, j, 3] + x[i+L[l]-1, j, 4] + sum2
                    )    

 

    
    #constraint 7
    
    for l in range(numShips):
        model.addConstr(
            gp.quicksum(s[i, j, l] for i in range(I) for j in range(J)) == S[l]
        )

    #constraint 8
    for i in range(I):
        for j in range(J):
            model.addConstr(
                gp.quicksum(x[i, j, k] for k in range(1, K)) + 
                (gp.quicksum(x[i+1, j+1, k] for k in range(1, K)) if i+1 < I and j+1 < J else 0)
                <= 1
            )

    #constraint 9
    for i in range(I):
        for j in range(J):
            model.addConstr(
                gp.quicksum(x[i, j, k] for k in range(1, K)) + 
                (gp.quicksum(x[i-1, j+1, k] for k in range(1, K)) if i > 0 and j+1 < J else 0)
                <= 1
            )



    #constraint 10
    for i in range(I):
        for j in range(J):
            model.addConstr(
                x[i, j, 1] <= 
                (x[i, j+1, 2] if j+1 < J else 0) + 
                (x[i, j+1, 5] if j+1 < J else 0)
            )

    #constraint 11
    for i in range(I):
        for j in range(1,J):
                model.addConstr(
                    x[i, j, 1] <= x[i, j-1, 0]
                )

    #constraint 12
    for i in range(I):
        for j in range(J-1):
                model.addConstr(
                    x[i, j, 2] <= x[i, j+1, 0]
                )
            
    #constraint 13
    for i in range(I):
        for j in range(J):
            model.addConstr(
                x[i, j, 2] <= 
                (x[i, j-1, 1] if j > 0 else 0) +  # Left-end (◀) to the left
                (x[i, j-1, 5] if j > 0 else 0)   # Middle part (■) to the left
            )

    #constraint 14
    for i in range(I):
        for j in range(J):
            model.addConstr(
                x[i, j, 3] <= 
                (x[i+1, j, 4] if i+1 < I else 0) +  # Right-end (▶) below
                (x[i+1, j, 5] if i+1 < I else 0)   # Middle part (■) below
            )

    #constraint 15
    for i in range(1,I):
        for j in range(J):
                model.addConstr(
                    x[i, j, 3] <= x[i-1, j, 0]
                )

    #constraint 16
    for i in range(I-1):
        for j in range(J):
                model.addConstr(
                    x[i, j, 4] <= x[i+1, j, 0]
                )

    #constraint 17
    for i in range(I):
        for j in range(J):
            model.addConstr(
                x[i, j, 4] <= 
                (x[i-1, j, 3] if i > 0 else 0) +  # Bottom-end (▼) above
                (x[i-1, j, 5] if i > 0 else 0)   # Middle part (■) above
            )

    #constraint 18
    for i in range(I):
        for j in range(J):
            model.addConstr(
                x[i, j, 5] <= 
                (x[i-1, j, 3] if i > 0 else 0) +  # Bottom-end (▼) above
                (x[i-1, j, 5] if i > 0 else 0) +  # Middle part (■) above
                (x[i, j+1, 2] if j+1 < J else 0) +  # Top-end (▲) to the right
                (x[i, j+1, 5] if j+1 < J else 0),   # Middle part (■) to the right
            )

    #constraint 19
    for i in range(I):
        for j in range(J):
            model.addConstr(
                x[i, j, 5] <= 
                (x[i-1, j, 3] if i > 0 else 0) +  # Bottom-end (▼) above
                (x[i-1, j, 5] if i > 0 else 0) +  # Middle part (■) above
                (x[i, j-1, 1] if j > 0 else 0) +  # Left-end (◀) to the top-left
                (x[i, j-1, 5] if j > 0 else 0)   # Middle part (■) to the top-left
            )

    #constraint 20
    for i in range(I):
        for j in range(J):
            model.addConstr(
                x[i, j, 5] <= 
                (x[i+1, j, 4] if i+1 < I else 0) +  # Right-end (▶) below
                (x[i+1, j, 5] if i+1 < I else 0) +  # Middle part (■) below
                (x[i, j-1, 1] if j > 0 else 0) +  # Left-end (◀) to the left
                (x[i, j-1, 5] if j > 0 else 0)   # Middle part (■) to the left
            )

    #constraint 21
    for i in range(I):
        for j in range(J):
            model.addConstr(
                x[i, j, 5] <= 
                (x[i+1, j, 4] if i+1 < I else 0) +  # Right-end (▶) below
                (x[i+1, j, 5] if i+1 < I else 0) +  # Middle part (■) below
                (x[i, j+1, 2] if j+1 < J else 0) +  # Top-end (▲) to the right
                (x[i, j+1, 5] if j+1 < J else 0)   # Middle part (■) to the right
            )

    #constraint 22
    for i in range(I):
        for j in range(1,J):
                model.addConstr(
                    x[i, j, 6] <= 1 - x[i, j-1, 6]
                )

    #constraint 23
    for i in range(1,I):
        for j in range(J):
                model.addConstr(
                    x[i, j, 6] <= 1 - x[i-1, j, 6]
            )

    # Solve the model
    model.setParam("Presolve", 0)
    model.optimize()

    symbol_map = {
    0: "≈",  # Empty cell
    1: "◀",  # Left-end of a ship
    2: "▶",  # Top-end of a ship
    3: "▲",  # Bottom-end of a ship
    4: "▼",  # Right-end of a ship
    5: "■",  # Middle of a ship
    6: "●"   # Single-cell ship
    }

    if model.status == GRB.OPTIMAL:
        # Create an empty grid
        grid = [[" " for _ in range(J)] for _ in range(I)]
        
        # Iterate over all cells to assign symbols
        for i in range(I):
            for j in range(J):
                for k in range(7):  # k in {0,1,2,3,4,5,6}
                    if x[i, j, k].x > 0.5:  # If variable is set to 1
                        grid[i][j] = symbol_map[k]
        
        # Print the grid
        print("\nSolution Grid:")
        for row in grid:
            print(" ".join(row))
    else:
        print("No optimal solution found.")
    


In [29]:
I = 6
J = 6
R = [3, 1, 3, 1, 2, 0]
C = [3, 0, 2, 3, 1, 1]
S = [3, 2, 1]
L = [1, 2, 3]
K = 7
numships = len(S)
T = {(i, j, k): 100 for i in range(0, I) for j in range(0, J) for k in range(K)}


symbol_map = {
0: "≈",  # Empty cell
1: "◀",  # Left-end of a ship
2: "▶",  # Top-end of a ship
3: "▲",  # Bottom-end of a ship
4: "▼",  # Right-end of a ship
5: "■",  # Middle of a ship
6: "●"   # Single-cell ship
}

# Predefined ship placements (T[i, j, k] = 1)
predefined_cells = {
    (1, 0, 6): 1,
    (2, 5, 6): 1,
}

# Apply predefined ships
for (i, j, k), value in predefined_cells.items():
    T[(i, j, k)] = value  # Set predefined symbols



solve_battleships(I, J, R, C, T, S, L)
len(L)


Set parameter Presolve to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 670 rows, 360 columns and 3171 nonzeros
Model fingerprint: 0x1e5f7108
Variable types: 0 continuous, 360 integer (360 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Variable types: 0 continuous, 360 integer (360 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

Solution Grid:
≈ ≈ ◀ ■ ▶ ≈
● ≈ ≈ ≈ ≈ ≈
≈ ≈ ◀ 

3

In [31]:
I = 6
J = 6
R = [3, 1, 3, 1, 1, 1]
C = [1, 3, 2, 2, 0, 2]
S = [3, 2, 1]
L = [1, 2, 3]
K = 7
numships = len(S)
T = {(i, j, k): 100 for i in range(0, I) for j in range(0, J) for k in range(K)}


symbol_map = {
0: "≈",  # Empty cell
1: "◀",  # Left-end of a ship
2: "▶",  # Top-end of a ship
3: "▲",  # Bottom-end of a ship
4: "▼",  # Right-end of a ship
5: "■",  # Middle of a ship
6: "●"   # Single-cell ship
}

# Predefined ship placements (T[i, j, k] = 1)
predefined_cells = {
    (2, 0, 0): 1,
}

# Apply predefined ships
for (i, j, k), value in predefined_cells.items():
    T[(i, j, k)] = value  # Set predefined symbols



solve_battleships(I, J, R, C, T, S, L)
len(L)


Set parameter Presolve to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 669 rows, 360 columns and 3170 nonzeros
Model fingerprint: 0x65de3fcc
Variable types: 0 continuous, 360 integer (360 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Variable types: 0 continuous, 360 integer (360 binary)

Root relaxation: objective 0.000000e+00, 160 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   63          -    0.00000      -     -    0s
     0     0    0.00000    0   26          

3

In [33]:
I = 8
J = 8
R = [3, 2, 2, 2, 4, 1, 2, 3]
C = [4, 2, 0, 4, 2, 3, 3, 1]
S = [3, 3, 2, 1]
L = [1, 2, 3, 4]
K = 7
numships = len(S)
T = {(i, j, k): 100 for i in range(0, I) for j in range(0, J) for k in range(K)}


symbol_map = {
0: "≈",  # Empty cell
1: "◀",  # Left-end of a ship
2: "▶",  # Top-end of a ship
3: "▲",  # Bottom-end of a ship
4: "▼",  # Right-end of a ship
5: "■",  # Middle of a ship
6: "●"   # Single-cell ship
}

# Predefined ship placements (T[i, j, k] = 1)
predefined_cells = {
    (0, 4, 5): 1, 
    (5, 3, 4): 1,




}

# Apply predefined ships
for (i, j, k), value in predefined_cells.items():
    T[(i, j, k)] = value  # Set predefined symbols



solve_battleships(I, J, R, C, T, S, L)


        



Set parameter Presolve to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1236 rows, 704 columns and 6086 nonzeros
Model fingerprint: 0xaf0ab546
Variable types: 0 continuous, 704 integer (704 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Variable types: 0 continuous, 704 integer (704 binary)

Root relaxation: objective 0.000000e+00, 226 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   45          -    0.00000      -     -    0s
     0     0    0.00000    0   48         

In [35]:
I = 8
J = 8
R = [4, 0, 3, 2, 3, 1, 3, 3]
C = [1, 4, 3, 1, 4, 0, 5, 1]
S = [3, 3, 2, 1]
L = [1, 2, 3, 4]
K = 7
numships = len(S)
T = {(i, j, k): 100 for i in range(0, I) for j in range(0, J) for k in range(K)}


symbol_map = {
0: "≈",  # Empty cell
1: "◀",  # Left-end of a ship
2: "▶",  # Top-end of a ship
3: "▲",  # Bottom-end of a ship
4: "▼",  # Right-end of a ship
5: "■",  # Middle of a ship
6: "●"   # Single-cell ship
}

# Predefined ship placements (T[i, j, k] = 1)
predefined_cells = {
    (0, 4, 2): 1, 
    (7, 0, 6): 1,




}

# Apply predefined ships
for (i, j, k), value in predefined_cells.items():
    T[(i, j, k)] = value  # Set predefined symbols



solve_battleships(I, J, R, C, T, S, L)


        



Set parameter Presolve to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1236 rows, 704 columns and 6086 nonzeros
Model fingerprint: 0xec82f070
Variable types: 0 continuous, 704 integer (704 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Variable types: 0 continuous, 704 integer (704 binary)

Root relaxation: objective 0.000000e+00, 272 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   62          -    0.00000      -     -    0s
     0     0    0.00000    0   48         

In [37]:
I = 10
J = 10
R = [2, 5, 1, 2, 1, 1, 2, 3, 3, 0]
C = [3, 1, 5, 0, 2, 0, 3, 2, 3, 1]
S = [4, 3, 2, 1]
L = [1, 2, 3, 4]
K = 7
numships = len(S)
T = {(i, j, k): 100 for i in range(0, I) for j in range(0, J) for k in range(K)}


symbol_map = {
0: "≈",  # Empty cell
1: "◀",  # Left-end of a ship
2: "▶",  # Top-end of a ship
3: "▲",  # Bottom-end of a ship
4: "▼",  # Right-end of a ship
5: "■",  # Middle of a ship
6: "●"   # Single-cell ship
}

# Predefined ship placements (T[i, j, k] = 1)
predefined_cells = {
    (0, 0, 3): 1, 
    (5, 6, 4): 1,
    (6, 0, 0): 1,
    (7, 9, 2): 1,




}

# Apply predefined ships
for (i, j, k), value in predefined_cells.items():
    T[(i, j, k)] = value  # Set predefined symbols



solve_battleships(I, J, R, C, T, S, L)



        



Set parameter Presolve to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1962 rows, 1100 columns and 9784 nonzeros
Model fingerprint: 0x5f0f3276
Variable types: 0 continuous, 1100 integer (1100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Variable types: 0 continuous, 1100 integer (1100 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.01 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

Solution Grid:
▲ ≈ ≈ ≈ ● ≈ ≈ ≈ ≈ ≈
▼ ≈ 

In [39]:
I = 10
J = 10
R = [3, 1, 6, 0, 1, 5, 0, 1, 1, 2]
C = [5, 0, 3, 2, 1, 2, 3, 1, 3, 0]
S = [4, 3, 2, 1]
L = [1, 2, 3, 4]
K = 7
numships = len(S)
T = {(i, j, k): 100 for i in range(0, I) for j in range(0, J) for k in range(K)}


symbol_map = {
0: "≈",  # Empty cell
1: "◀",  # Left-end of a ship
2: "▶",  # Top-end of a ship
3: "▲",  # Bottom-end of a ship
4: "▼",  # Right-end of a ship
5: "■",  # Middle of a ship
6: "●"   # Single-cell ship
}

# Predefined ship placements (T[i, j, k] = 1)
predefined_cells = {
    (5, 4, 2): 1, 
    (7, 6, 6): 1,
    (8, 2, 3): 1,
    (9, 8, 0): 1,




}

# Apply predefined ships
for (i, j, k), value in predefined_cells.items():
    T[(i, j, k)] = value  # Set predefined symbols



solve_battleships(I, J, R, C, T, S, L)


        



Set parameter Presolve to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1962 rows, 1100 columns and 9784 nonzeros
Model fingerprint: 0x64bb173a
Variable types: 0 continuous, 1100 integer (1100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Variable types: 0 continuous, 1100 integer (1100 binary)

Root relaxation: objective 0.000000e+00, 216 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   12          -    0.00000      -     -    0s
     0     0    0.00000    0   10    

In [41]:
I = 15
J = 15
R = [4, 0, 6, 1, 0, 2, 2, 3, 2, 4, 3, 1, 5, 1, 1]
C = [0, 7, 3, 1, 4, 5, 0, 1, 1, 3, 2, 2, 3, 2, 1]
S = [5, 4, 3, 2, 1]
L = [1, 2, 3, 4, 5]
K = 7
numships = len(S)
T = {(i, j, k): 100 for i in range(0, I) for j in range(0, J) for k in range(K)}


symbol_map = {
0: "≈",  # Empty cell
1: "◀",  # Left-end of a ship
2: "▶",  # Top-end of a ship
3: "▲",  # Bottom-end of a ship
4: "▼",  # Right-end of a ship
5: "■",  # Middle of a ship
6: "●"   # Single-cell ship
}

# Predefined ship placements (T[i, j, k] = 1)
predefined_cells = {
    (0, 2, 1): 1, 
    (0, 9, 6): 1,
    (2, 13, 5): 1,
    (7, 13, 0): 1,
    (8, 5, 3): 1,
    (9, 8, 5): 1,
    (10, 12, 2): 1,
    (11, 5, 5): 1,
    (13, 1, 5): 1,




}

# Apply predefined ships
for (i, j, k), value in predefined_cells.items():
    T[(i, j, k)] = value  # Set predefined symbols



solve_battleships(I, J, R, C, T, S, L)


        



Set parameter Presolve to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4634 rows, 2700 columns and 24193 nonzeros
Model fingerprint: 0x5eb0ba7f
Variable types: 0 continuous, 2700 integer (2700 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Variable types: 0 continuous, 2700 integer (2700 binary)

Root relaxation: objective 0.000000e+00, 416 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    6          -    0.00000      -     -    0s
     0     0    0.00000    0   32   

In [43]:
I = 15
J = 15
R = [3, 1, 4, 0, 7, 1, 2, 1, 1, 3, 5, 5, 1, 1, 0]
C = [4, 1, 2, 2, 5, 2, 1, 3, 3, 4, 2, 2, 1, 3, 0]
S = [5, 4, 3, 2, 1]
L = [1, 2, 3, 4, 5]
K = 7
numships = len(S)
T = {(i, j, k): 100 for i in range(0, I) for j in range(0, J) for k in range(K)}


symbol_map = {
0: "≈",  # Empty cell
1: "◀",  # Left-end of a ship
2: "▶",  # Top-end of a ship
3: "▲",  # Bottom-end of a ship
4: "▼",  # Right-end of a ship
5: "■",  # Middle of a ship
6: "●"   # Single-cell ship
}

# Predefined ship placements (T[i, j, k] = 1)
predefined_cells = {
    (0, 5, 2): 1, 
    (1, 11, 3): 1,
    (4, 1, 6): 1,
    (5, 4, 5): 1,
    (7, 9, 6): 1,
    (8, 0, 3): 1,
    (9, 12, 1): 1,
    (12, 13, 4): 1,
    (13, 5, 6): 1,




}

# Apply predefined ships
for (i, j, k), value in predefined_cells.items():
    T[(i, j, k)] = value  # Set predefined symbols



solve_battleships(I, J, R, C, T, S, L)


        



Set parameter Presolve to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4634 rows, 2700 columns and 24193 nonzeros
Model fingerprint: 0x280b3775
Variable types: 0 continuous, 2700 integer (2700 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Variable types: 0 continuous, 2700 integer (2700 binary)

Root relaxation: objective 0.000000e+00, 317 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   20          -    0.00000      -     -    0s
     0     0    0.00000    0   34   

In [23]:
I = 15
J = 10
R = [2, 1, 2, 0, 2, 0, 7, 1, 6, 1, 2, 2, 5, 2, 2]
C = [4, 4, 3, 3, 5, 2, 3, 3, 1, 7]
S = [5, 4, 3, 2, 1]
L = [1, 2, 3, 4, 5]
K = 7
numships = len(S)
T = {(i, j, k): 100 for i in range(0, I) for j in range(0, J) for k in range(K)}


symbol_map = {
0: "≈",  # Empty cell
1: "◀",  # Left-end of a ship
2: "▶",  # Top-end of a ship
3: "▲",  # Bottom-end of a ship
4: "▼",  # Right-end of a ship
5: "■",  # Middle of a ship
6: "●"   # Single-cell ship
}

# Predefined ship placements (T[i, j, k] = 1)
predefined_cells = {
    (0, 4, 0): 1, 
    (0, 8, 0): 1,
    (2, 3, 2): 1,
    (4, 1, 0): 1,
    (4, 6, 6): 1,
    (4, 8, 0): 1,
    (6, 9, 0): 1,
    (8, 4, 0): 1,
    (8, 7, 0): 1,
    (8, 9, 5): 1,
    (10, 1, 0): 1,
    (12, 7, 0): 1,
    (14, 7, 2): 1,

}

# Apply predefined ships
for (i, j, k), value in predefined_cells.items():
    T[(i, j, k)] = value  # Set predefined symbols



solve_battleships(I, J, R, C, T, S, L)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3048 rows, 1800 columns and 15707 nonzeros
Model fingerprint: 0xf98ffb90
Variable types: 0 continuous, 1800 integer (1800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Presolve removed 3048 rows and 1800 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.02 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

Solution Grid:
◀ ▶ ≈ ≈ ≈ ≈ ≈ ≈ ≈ ≈
≈ ≈ ≈ ≈ ≈ ≈ ≈ ≈ ● ≈
≈ ≈ ◀ ▶ ≈ ≈ ≈ ≈ 

In [25]:
I = 15
J = 15
R = [1, 3, 3, 1, 1, 4, 3, 1, 1, 4, 3, 3, 1, 2, 4]
C = [2, 3, 4, 1, 1, 2, 1, 5, 1, 4, 1, 2, 5, 3, 0]
S = [5, 4, 3, 2, 1]
L = [1, 2, 3, 4, 5]
K = 7
numships = len(S)
T = {(i, j, k): 100 for i in range(0, I) for j in range(0, J) for k in range(K)}


symbol_map = {
0: "≈",  # Empty cell
1: "◀",  # Left-end of a ship
2: "▶",  # Top-end of a ship
3: "▲",  # Bottom-end of a ship
4: "▼",  # Right-end of a ship
5: "■",  # Middle of a ship
6: "●"   # Single-cell ship
}

# Predefined ship placements (T[i, j, k] = 1)
predefined_cells = {
    (1, 13, 2): 1, 
    (3, 7, 5): 1,
    (5, 0, 3): 1,
    (5, 7, 4): 1,
    (5, 9, 3): 1,
    (7, 2, 5): 1,
    (10, 4, 5): 1,
    (13, 1, 5): 1,


}

# Apply predefined ships
for (i, j, k), value in predefined_cells.items():
    T[(i, j, k)] = value  # Set predefined symbols



solve_battleships(I, J, R, C, T, S, L)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4633 rows, 2700 columns and 24192 nonzeros
Model fingerprint: 0xae67a8b4
Variable types: 0 continuous, 2700 integer (2700 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 4516 rows and 2596 columns
Presolve time: 0.03s
Presolved: 117 rows, 104 columns, 646 nonzeros
Variable types: 0 continuous, 104 integer (104 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.02 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best b